In [2]:
import pandas as pd

In [2]:
pre_pd = pd.read_csv('../data/raw/domain.csv')
pre_pd.head()

,id,promoType,price,hasVideo,agentNames,brandName,addressStreet,addressSuburb,addressState,addressPostcode,addressLat,addressLng,num_bath,type,formatted,isRural,landSize,Retirement
0,15230553,premiumplus,$500 per week,False,Ismaeel Hussain,Samma Real Estate,322/23 Batman Street,WEST MELBOURNE,VIC,3003,-37.810738,144.95226,1.0,ApartmentUnitFlat,Apartment / Unit / Flat,False,0.0,False
1,15760997,premiumplus,$540 per Week,True,Conny Yang,Waterdale Property Agent Pty Ltd,2601/65 Dudley Street,WEST MELBOURNE,VIC,3003,-37.810135,144.95040,1.0,ApartmentUnitFlat,Apartment / Unit / Flat,False,0.0,False
2,15975970,standardpp,$650.00,False,Ismaeel Hussain,Samma Real Estate,1212/15-31 Batman Street,WEST MELBOURNE,VIC,3003,-37.810757,144.95224,2.0,ApartmentUnitFlat,Apartment / Unit / Flat,False,0.0,False
3,16668342,premiumplus,$560 per week,False,Ismaeel Hussain,Samma Real Estate,307/33 Batman Street,WEST MELBOURNE,VIC,3003,-37.810860,144.95183,1.0,ApartmentUnitFlat,Apartment / Unit / Flat,False,0.0,False
4,16803052,standardpp,$600,False,Caroline Hardeman,Ray White Williamstown,2/18 Ireland Street,WEST MELBOURNE,VIC,3003,-37.807606,144.94562,1.0,ApartmentUnitFlat,Apartment / Unit / Flat,False,0.0,False


In [3]:
postcode_counts = pre_pd['addressPostcode'].value_counts()
valid_postcodes = postcode_counts[postcode_counts > 90].index

# Step 2: Filter the original dataframe based on the valid postcodes
new_df = pre_pd[pre_pd['addressPostcode'].isin(valid_postcodes)]

# Display the filtered dataframe
print(len(new_df))

6992


In [4]:
condition = new_df['addressPostcode'] == 3000

# Step 1: Filter rows based on the condition
filtered_df = new_df[condition]

fraction_to_delete = 0.8
rows_to_delete = filtered_df.sample(frac=fraction_to_delete, random_state=42).index

# Step 3: Drop the selected rows from the original DataFrame
df_after_deletion = new_df.drop(rows_to_delete)

df_after_deletion.to_csv('../data/curated/domain_del.csv', index=False)

## A. Pre-process on Price column

In [3]:
df = pd.read_csv("../data/curated/domain_del.csv", encoding='latin1')

In [4]:
df.columns

Index(['id', 'promoType', 'price', 'hasVideo', 'agentNames', 'brandName',
       'addressStreet', 'addressSuburb', 'addressState', 'addressPostcode',
       'addressLat', 'addressLng', 'num_bath', 'type', 'formatted', 'isRural',
       'landSize', 'Retirement'],
      dtype='object')

In [5]:
# 清理之前的dataset numbers
price_columns_before = df['id'].count()

# inspect the price columns
df['price']

0       $500 per week
1       $540 per Week
2             $650.00
3       $560 per week
4                $600
            ...      
4909    $470 per week
4910             $600
4911             $575
4912    $530 per week
4913             $530
Name: price, Length: 4914, dtype: object

## Step 1 : Extract the price by Regax

In [12]:
# 过滤掉只含有文字列 提取数字
df['price'] = df['price'].str.extract(r'(\d+)')

# transfer to float
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Homes Victoria rents for the third quarter of 2023 to filter the records not in $170 ~ $2400,
# and delete missing values caused by string values (e.g. Inspection only) or missed records.
df = df[(df['price'] >= 170) & (df['price'] <= 2400)]

In [6]:
df['price']

0       $500 per week
1       $540 per Week
2             $650.00
3       $560 per week
4                $600
            ...      
4909    $470 per week
4910             $600
4911             $575
4912    $530 per week
4913             $530
Name: price, Length: 4914, dtype: object

In [7]:
# whole dataset instances
print(f"Before cleaning on price columns, there are {price_columns_before} records in the main dataset.")
print(f"After cleaning on price columns, there are {df['id'].count()} records in the main dataset.")

Before cleaning on price columns, there are 4914 records in the main dataset.
After cleaning on price columns, there are 4914 records in the main dataset.


## Save the dataset

In [8]:
df.to_csv('../data/curated/Price_clean_domain.csv', index=False)

Drop Null value

In [12]:
position_df = pd.read_csv("../data/curated/Non_longi_lati_remove_domain.csv")

In [15]:
position_df = position_df.dropna(subset = ['addressLat'])

In [16]:
position_df.to_csv('../data/curated/None_longi_lati_remove_domain.csv', index=False)